In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Dataset_Skripsi_Processed.csv'  # Path absolut
df = pd.read_csv(file_path)

In [3]:
# Menampilkan data pertama untuk memastikan dataset terbaca dengan benar
pd.set_option('display.max_rows', None)
print(df)

    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
0             P01         G01  0.0  0.0        0.0       0.0
1             P01         G02  0.6  1.0        0.8       0.8
2             P01         G03  0.0  0.0        0.0       0.0
3             P01         G04  0.0  0.0        0.0       0.0
4             P01         G05  0.0  0.0        0.0       0.0
5             P01         G06  0.0  0.0        0.0       0.0
6             P01         G07  0.0  0.0        0.0       0.0
7             P01         G08  0.0  0.0        0.0       0.0
8             P01         G09  1.0  0.8        0.9       1.0
9             P01         G10  0.8  1.0        0.9       1.0
10            P01         G11  0.6  1.0        0.8       0.8
11            P01         G12  0.6  1.0        0.8       0.8
12            P01         G13  0.0  0.0        0.0       0.0
13            P01         G14  0.0  0.0        0.0       0.0
14            P01         G15  0.0  0.0        0.0       0.0
15            P01       

In [4]:
# Filter data untuk P04 saja
df_P04 = df[df['kode_penyakit'] == 'P04']  # Ambil data hanya untuk P04

In [5]:
# Menampilkan data yang diambil untuk P04
print("Data untuk P04:")
print(df_P04)

Data untuk P04:
    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
93            P04         G01  0.0  0.0        0.0       0.0
94            P04         G02  0.0  0.0        0.0       0.0
95            P04         G03  0.0  0.0        0.0       0.0
96            P04         G04  0.0  0.0        0.0       0.0
97            P04         G05  0.0  0.0        0.0       0.0
98            P04         G06  0.0  0.0        0.0       0.0
99            P04         G07  0.0  0.0        0.0       0.0
100           P04         G08  0.0  0.0        0.0       0.0
101           P04         G09  0.0  0.0        0.0       0.0
102           P04         G10  0.0  0.0        0.0       0.0
103           P04         G11  0.0  0.0        0.0       0.0
104           P04         G12  0.0  0.0        0.0       0.0
105           P04         G13  0.0  0.0        0.0       0.0
106           P04         G14  0.4  0.0        0.2       0.2
107           P04         G15  0.0  0.0        0.0       0.0
108     

In [6]:
# Set seed for reproducibility
np.random.seed(42)

## Inisialisasi Parameter

In [7]:
# Parameter PSO tetap
w_max = 0.8  
w_min = 0.2
c1 = 1.5
c2 = 1.5
n_particles = 95  

## Fungsi Fitness

In [8]:
# Fungsi fitness untuk mengukur sejauh mana posisi partikel mendekati cf_pakar
def fitness(position, cf_pakar):
    return np.mean((cf_pakar - position) ** 2, axis=1)  # MSE 

## Implementasi

In [9]:
# Fungsi untuk menjalankan PSO dengan parameter bobot inersia yang berbeda untuk setiap penyakit
def run_pso(cf_pakar, n_iterations):
    fitness_best_results = []
    
    for trial in range(1, 11):  # 10 uji coba
        # Inisialisasi kecepatan dan posisi partikel secara acak
        position = np.random.uniform(0.0, 1.0, (n_particles, len(cf_pakar)))  # Posisi partikel
        velocity = np.random.uniform(-0.1, 0.1, (n_particles, len(cf_pakar)))  # Kecepatan partikel

        # Inisialisasi pbest dan gbest
        pbest = position.copy()  # Set pbest awal sama dengan posisi partikel
        pbest_score = np.array([np.mean(np.abs(cf_pakar - p)) for p in position])  # Nilai fitness awal untuk pbest
        gbest = pbest[np.argmin(pbest_score)]  # gbest adalah posisi dengan nilai fitness terbaik
        gbest_score = np.min(pbest_score)  # Nilai fitness terbaik untuk gbest

        # Iterasi PSO
        for iteration in range(n_iterations):
            w = w_max - (w_max - w_min) * (iteration / n_iterations)  # Interpolasi w antara w_max dan w_min

            # Update posisi dan kecepatan setiap partikel
            for i in range(n_particles):
                r1 = np.random.rand(len(cf_pakar))  # Random number untuk C1
                r2 = np.random.rand(len(cf_pakar))  # Random number untuk C2

                # Update kecepatan
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - position[i]) + c2 * r2 * (gbest - position[i])

                # Update posisi
                position[i] += velocity[i]

                # Pastikan posisi berada dalam rentang 0 dan 1
                position[i] = np.clip(position[i], 0.0, 1.0)

            # Evaluasi fitness setiap partikel
            score = fitness(position, cf_pakar)

            # Update pbest dan gbest
            for i in range(n_particles):
                if score[i] < pbest_score[i]:
                    pbest[i] = position[i]  # Update pbest dengan posisi partikel yang lebih baik
                    pbest_score[i] = score[i]

            # Update gbest jika ditemukan pbest yang lebih baik
            gbest_score_new = np.min(pbest_score)
            if gbest_score_new < gbest_score:
                gbest = pbest[np.argmin(pbest_score)]  # Update gbest dengan posisi pbest terbaik
                gbest_score = gbest_score_new

        # Simpan hasil fitness terbaik untuk setiap uji coba
        fitness_best_results.append(gbest_score)

    # Mengembalikan hasil fitness terbaik untuk 10 uji coba
    return fitness_best_results

In [10]:
# Daftar penyakit yang ingin diuji (hanya P04 untuk sekarang)
penyakit_codes = ['P04']  # Fokus pada P04 saja

In [11]:
# Menyimpan hasil uji coba per penyakit
results = []

In [12]:
# Kombinasi bobot inersia yang diuji
n_iterations_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

In [13]:
# Fokus pada P04
for penyakit in penyakit_codes:
    cf_pakar = df[df['kode_penyakit'] == penyakit]['cf_pakar'].values
    for n_iterations in n_iterations_values:
        # Jalankan PSO dan simpan hasil per percobaan
            fitness_results = run_pso(cf_pakar, n_iterations)
            
            # Simpan hasil per percobaan (trial) untuk n_iterations yang diuji
            for trial_num, fitness_score in enumerate(fitness_results, 1):
                results.append({
                    'Penyakit': penyakit,
                    'n_iterations': n_iterations,
                    'Percobaan': trial_num,
                    'Fitness Terbaik': fitness_score
                })
            

In [14]:
# Membuat DataFrame untuk menampilkan hasil
df_results = pd.DataFrame(results)

pd.set_option('display.max_rows', 1000)

In [15]:
# Menampilkan hasil tabel per percobaan
print(df_results)

    Penyakit  n_iterations  Percobaan  Fitness Terbaik
0        P04             5          1     6.920214e-02
1        P04             5          2     5.731032e-02
2        P04             5          3     5.073835e-02
3        P04             5          4     6.328864e-02
4        P04             5          5     5.968172e-02
5        P04             5          6     3.037911e-02
6        P04             5          7     7.856446e-02
7        P04             5          8     5.273030e-02
8        P04             5          9     4.829220e-02
9        P04             5         10     5.782914e-02
10       P04            10          1     1.529967e-02
11       P04            10          2     1.450083e-03
12       P04            10          3     3.522999e-03
13       P04            10          4     5.645087e-03
14       P04            10          5     5.840651e-03
15       P04            10          6     4.307173e-02
16       P04            10          7     4.543368e-03
17       P

In [16]:
# Menghitung rata-rata fitness terbaik untuk setiap kombinasi ωmax dan ωmin
avg_fitness_results = df_results.groupby(['Penyakit', 'n_iterations'])['Fitness Terbaik'].mean().reset_index()

In [17]:
# Menampilkan kombinasi terbaik per penyakit berdasarkan rata-rata fitness
print("\nKombinasi n_iterations terbaik per penyakit:")
print(avg_fitness_results)


Kombinasi n_iterations terbaik per penyakit:
   Penyakit  n_iterations  Fitness Terbaik
0       P04             5         0.056802
1       P04            10         0.011975
2       P04            15         0.007974
3       P04            20         0.012565
4       P04            25         0.010626
5       P04            30         0.011741
6       P04            35         0.009032
7       P04            40         0.008387
8       P04            45         0.010575
9       P04            50         0.016903
10      P04            55         0.017935
11      P04            60         0.020258
12      P04            65         0.010323
13      P04            70         0.004516
14      P04            75         0.011484
15      P04            80         0.008645
16      P04            85         0.009161
17      P04            90         0.010065
18      P04            95         0.010968
19      P04           100         0.008774


In [18]:
# Jika ingin menambahkan header dan data ke file yang sudah ada (append):
df_results.to_csv('pengujian_ukuran iterasi_P04.csv', mode='a', header=False, index=False)
